## Implement the geometric product for multivectors

For example, $(1+e_1e_2 + e_1e_2e_3) (1+e_1+e_3) = 1 + e_1 - e_2 + e_3 + 2 e_1 e_2 + e_2e_3 + 2 e_1e_2e_3$

Overview of the interface:

There are several classes currently implemented:
- BasisBlade, which stores the basic blades composed of basis vectors (but without any sign information)
- Blade, which pairs BasisBlade with a weight and supports addition/subtraction operations
- MultivectorExplicit, which is made of a collection/sum of Blade objects
- GnObj, which stores the lookup tables to work with multivectors in the flattened coordinates/indices

In [1]:
import numpy as np
import copy
from numbers import Number
from typing import Iterable, Callable
import itertools
rng = np.random.default_rng(0)

In [2]:
# See https://docs.python.org/3/library/itertools.html
# for the original code based on itertolls
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

In [3]:
class BasisBlade():
    """Can store blades in the form of e1 e2 e4 e7 (for example)"""
    def __init__(self, *arg_idcs, one_indexed=True):
        """Takes idcs as an iterable with indices (e.g., (0,1,2,6)) or None value for a scalar
        Store internally as 0-indexed but allow for 1-indexed inputs by default

        Note: automatically rearranges indices as the BasisBlade class has no concept of sign!
        """
        self.is_scalar   = len(arg_idcs) == 0 or arg_idcs[0] is None or isinstance(arg_idcs[0], tuple)
        self.one_indexed = one_indexed
        if self.is_scalar:
            self.idcs = np.array([])
        else:
            self.idcs = np.sort(np.unique(np.array(arg_idcs).astype(int)))
            if one_indexed:
                self.idcs = self.idcs - 1
        self.k = self.idcs.size
        self.n_lower_bound = 1 if self.is_scalar else np.max(self.idcs)

    def __str__(self):
        """More visually-appealing representation"""
        if self.is_scalar:
            return f"(1)"
        str_rep = ""
        for idx in self.idcs:
            str_rep += f"e{idx+1}"
        return str_rep
    __repr__ = __str__
    # def __repr__(self):
    #     """Potentially more useful info for debugging"""
    #     return f"Basis blade with idcs={self.idcs}"
    
    def __eq__(self, other):
        if other is None:
            return False
        assert isinstance(other, BasisBlade)
        slen = len(self.idcs)
        olen = len(other.idcs)
        if slen == 0 or olen == 0:
            return slen == olen
        return (slen == olen) and np.all(self.idcs == other.idcs)
    
    def __mul__(self, other, skip_clean=False):
        if isinstance(other, Number):
            res = Blade(other, self)
        elif isinstance(other, BasisBlade):
            res = Blade(1, self)*Blade(1, other)
        elif isinstance(other, Blade):
            res = Blade(1, self)*other
        else:
            raise ValueError(f"Not implemented: BasisBlade.__mul__ passed arg of type {type(other)}")
        return clean_mvec(res, skip=skip_clean)

    def __rmul__(self, other):
        if isinstance(other, Number):
            return clean_mvec(Blade(other, self))
        else:
            raise ValueError(f"Not supported: BasisBlade.__rmul__ passed arg of type {type(other)}")

    def __neg__(self):
        return Blade(-1, self)
    
    def __add__(self, other, skip_clean=False):
        if isinstance(other, Number):
            res = Blade(1, self) + Blade(other, None)
        elif isinstance(other, BasisBlade):
            res = Blade(1, self) + Blade(1, other)
        elif isinstance(other, MultivectorExplicit):
            res = MultivectorExplicit([Blade(1, self)]) + other
        else:
            raise ValueError(f"Not implemented: BasisBlade.__add__ passed arg of type {type(other)}")
        return clean_mvec(res, skip=skip_clean)

    __radd__ = __add__ # commutative

    def to_internal_idx(self, n=None):
        """Can just use lexicographic order for internal reductions"""
        # k is the grade of this blade..
        if self.is_scalar:
            val = 0
        else:
            n = self.n_lower_bound if n is None else n
            dense_idcs = np.zeros(n+1, dtype=bool)
            dense_idcs[self.idcs] = 1
            # print(f"dense idcs: {dense_idcs}")
            # val = np.sum(np.left_shift(dense_idcs, np.arange(n, -1, -1)))
            val = np.sum(np.left_shift(dense_idcs, np.arange(n+1)))
        return 2**(self.k) + val # not a dense indexing but would allow for the appropriate ordering I think

Scalar = BasisBlade()

# Parity sign operator
psgn = lambda k: 1 if (k%2 == 0) else -1


class Blade():
    """Store the weight here to make it easier to absorb sign flips"""

    def __init__(self, weight, basis_blade):
        """Stores a weight in addition to the standard stuff"""
        self.weight = weight
        self.basis_blade = basis_blade

    def __str__(self):
        return f"{str(np.array(self.weight))} {str(self.basis_blade)}"
    __repr__ = __str__
    # def __repr__(self):
    #     return f"Blade with w={self.weight}, idcs={self.basis_blade.idcs}"
    def __eq__(self, other):
        return (self.weight == other.weight) and (self.basis_blade == other.basis_blade)
    def __neg__(self):
        return Blade(-self.weight, self.basis_blade)

    def __rmul__(self, other):
        """Should only need to perform a multiplication with a scalar"""
        if isinstance(other, Number):
            return clean_mvec(Blade(self.weight*other, self.basis_blade))
        else:
            raise ValueError(f"Not supported: Blade.__rmul__ passed arg of type {type(other)}")
    def __sub__(self, other):
        return clean_mvec(self + other.__neg__())
    def __rsub__(self, other):
        return clean_mvec(other + self.__neg__())

    def __mul__(self, other, skip_clean=False):
        """Perform a multiplication to get another blade
        Effectively merge the two sorted lists while counting the number
        of indices from the first blade exceeding the other blade (when relevant)
        """
        sign_adj = 1 # sign adjustment
        sidcs = self.basis_blade.idcs
        if isinstance(other, Number):
            res = Blade(self.weight*other, self.basis_blade)
        elif isinstance(other, BasisBlade):
            res = self * Blade(1, other)
        elif isinstance(other, Blade):
            oidcs = other.basis_blade.idcs
            slen  = sidcs.shape[0]
            olen  = oidcs.shape[0]
            if slen == 0:
                res = Blade(self.weight*other.weight*sign_adj, other.basis_blade)
                return clean_mvec(res)
            elif olen == 0:
                res = Blade(self.weight*other.weight*sign_adj, self.basis_blade)
                return clean_mvec(res)
            new_idcs = []
            si = 0
            oi = 0
            while (si < slen) and (oi < olen):
                k = si+oi
                # print(f"k={k}; new_idcs={new_idcs}; sidcs={sidcs[si:]}; oidcs={oidcs[oi:]}")
                if oidcs[oi] < sidcs[si]:
                    # Shift the idx from other blade up
                    new_idcs.append(oidcs[oi])
                    num_after = slen - si # check this..
                    sign_adj *= psgn(num_after)
                    oi += 1
                elif oidcs[oi] > sidcs[si]:
                    # Shift the idx from self's blade up; no sign adjustment needed
                    new_idcs.append(sidcs[si])
                    si += 1
                else: #  oidcs[oi] == sidcs[si]:
                    # in case of a tie, advance both but update the sign counter
                    num_after = slen - (si+1) # for safety..
                    sign_adj *= psgn(num_after) # make the sign adjustments by parity required
                    oi += 1
                    si += 1
            if si < slen:
                new_idcs = new_idcs + list(sidcs[si:])
            elif oi < olen:
                new_idcs = new_idcs + list(oidcs[oi:])
            # print(f"End; new_idcs={new_idcs}; sidcs={sidcs[si:]}; oidcs={oidcs[oi:]}")
            # print(f"Sign adjustment: {sign_adj}")
            res = Blade(self.weight*other.weight*sign_adj, BasisBlade(*new_idcs, one_indexed=False))
        else:
            raise ValueError(f"Not implemented: Blade.__mul__ passed arg of type {type(other)}")
        return clean_mvec(res, skip=skip_clean)

    def __add__(self, other, skip_clean=False):
        """Get a multivector if the basis vectors don't match"""
        if isinstance(other, Number):
            res =  self + Blade(other, BasisBlade(None)) # promote scalar to basis blade type
        elif isinstance(other, BasisBlade):
            res =  self + Blade(1, other)
        elif isinstance(other, Blade):
            """Now can do some comparisons"""
            if self.basis_blade == other.basis_blade:
                res =  Blade(self.weight + other.weight, self.basis_blade)
            else:
                res =  MultivectorExplicit([self, other])
        elif isinstance(other, Iterable):
            res =  self + MultivectorExplicit(other)
        elif isinstance(other, MultivectorExplicit):
            res =  MultivectorExplicit([self]) + other
        else:
            raise ValueError(f"Not implemented (yet): Blade.__add__ passed arg of type {type(other)}")
        return clean_mvec(res, skip=skip_clean)

    __radd__ = __add__ # commutative

    def __sub__(self, other):
        res = self + other.__neg__()
        return clean_mvec(res)
    def __rsub__(self, other):
        return clean_mvec(other + self.__neg__())

    def hashable_idcs(self):
        return tuple(self.basis_blade.idcs)

def OrdBlade(*arg_idcs, one_indexed=True):
    """If we want to input an signed/ordered version of the basis blades (still stored in sorted order)
    Slightly misleading capitalization as this returns a Blade type rather than being its own type
    """
    if len(arg_idcs) == 0 or arg_idcs[0] is None:
        parity_sign = 1 # give it as a None
        blade_idcs = None
    else:
        idcs = np.array(arg_idcs).astype(int)
        so   = np.argsort(idcs, kind='stable') # avoid spurious transpositions if there are repeats
        inversion_mat = so[:,np.newaxis] > so[np.newaxis, :]
        inversion_num = np.sum(np.triu(inversion_mat)) # count inversions for perm parity
        parity_sign = psgn(inversion_num)
        sorted_idcs = idcs[so]

        unique_idcs, counts = np.unique(sorted_idcs, return_counts=True)
        blade_idcs  = unique_idcs[counts%2 != 0]
    return Blade(parity_sign, BasisBlade(blade_idcs, one_indexed=one_indexed))

def reduce_blades(blades):
    """Helper function to reduce any repeats or zeroes
    For now, just use a cheap internal index until I figure out the visually meaningful version..
    """
    nonzero_blades = [blade for blade in blades if blade.weight != 0]
    nblen = len(nonzero_blades)
    n_max = np.max([blade.basis_blade.n_lower_bound for blade in nonzero_blades])
    so = np.argsort([blade.basis_blade.to_internal_idx(n_max) for blade in nonzero_blades])
    sorted_blades = [nonzero_blades[so[i]] for i in range(nblen)]

    last_basis_blade = None
    reduced_blades = []
    # ri = 0
    for si, sblade in enumerate(sorted_blades):
        if sblade.basis_blade != last_basis_blade:
            new_blade = Blade(sblade.weight, sblade.basis_blade) # make a new object
            last_basis_blade = sblade.basis_blade
            reduced_blades.append(new_blade)
        else:
            reduced_blades[-1].weight += sblade.weight
    return reduced_blades

def clean_mvec(mvec, skip=False):
    """Sends a zero blade to the zero scalar for simplicity;
    also sends a single-blade multivector to a blade (possibly the zero scalar)
    Takes any type
    """
    if skip:
        return mvec
    if isinstance(mvec, BasisBlade):
        res = mvec
    elif isinstance(mvec, Blade):
        res =  mvec if mvec.weight != 0 else Blade(0, Scalar)
    elif isinstance(mvec, Iterable):
        red_blades = reduce_blades(mvec)
        if len(red_blades) == 1:
            res = clean_mvec(red_blades[0]) # sends as a Blade object (which is checked for being zero)
        else:
            res = red_blades
    elif isinstance(mvec, MultivectorExplicit):
        cleaned_blades = clean_mvec(mvec.blades)
        if isinstance(cleaned_blades, Iterable):
            # assume it's a list of blades..
            res = MultivectorExplicit(cleaned_blades)
        else:
            res = cleaned_blades
    else:
        raise ValueError(f"(unsupported) clean_blade received argument with type {type(mvec)}")
    return res

class MultivectorExplicit():
    """Explicitly store the multivector as a weighted sum of the basis blades
    Could just as well store as a list of blades, but this provides a slightly lighter interface
    """
    def __init__(self, blades):
        """Takes in iterables of weights and relevant basis blades
        """
        self.blades = reduce_blades(blades) # don't call clean since that might not give a list of blades

    def __str__(self):
        string = ""
        for i, blade in enumerate(self.blades):
            w  = blade.weight
            bb = blade.basis_blade
            if w == 0:
                continue
            if i > 0:
                string += " + " if w > 0 else " - "
            if bb == BasisBlade():
                string += f"{str(np.abs(w))}"
            else:
                string += f"{str(np.abs(w))} {str(bb)}"
        return string
    __repr__ = __str__
    
    def __add__(self, other, skip_clean=False):
        if isinstance(other, Number):
            other_as_blade_list = [Blade(other, Scalar)]
        elif isinstance(other, BasisBlade):
            other_as_blade_list = [Blade(1, other)]
        elif isinstance(other, Blade):
            other_as_blade_list = [other]
        elif isinstance(other, MultivectorExplicit):
            other_as_blade_list = other
        # print(f">> {other_as_blade_list}")
        res = MultivectorExplicit(self.blades + other_as_blade_list)
        return clean_mvec(res, skip=skip_clean)

    __radd__ = __add__

    def __rmul__(self, other):
        if isinstance(other, Number):
            return MultivectorExplicit([[other*blade for blade in self.blades]])
        else:
            raise ValueError(f"Not implemented (yet): MultivectorExplicit.__rmul__ passed arg of type {type(other)}")

    def __mul__(self, other, skip_clean=False):
        """Basically handle by a cartesian product and a reduction"""
        if isinstance(other, Number):
            res = MultivectorExplicit([sb.__mul__(other, skip_clean=True) for sb in self.blades])
        elif isinstance(other, BasisBlade):
            res = self * Blade(1, other)
        elif isinstance(other, Blade):
            interactions = [sb.__mul__(other, skip_clean=True) for sb in self.blades] # list of blades
            res = Multivector(interactions)
        elif isinstance(other, MultivectorExplicit):
            all_interactions = [sb.__mul__(ob, skip_clean=True) for sb in self.blades for ob in other.blades] # list of blades
            res = MultivectorExplicit(all_interactions)
        else:
            raise ValueError(f"Not implemented (yet): MultivectorExplicit.__rmul__"
                             f" passed arg of type {type(other)}")
        return clean_mvec(res, skip=skip_clean)

In [4]:
print(MultivectorExplicit([OrdBlade(1,2,3), OrdBlade(1,2,3)]))

2 e1e2e3


In [5]:
OrdBlade(1,2,3) + OrdBlade(1,3,2)

0 (1)

In [6]:
# BasisBlade type captures the basic geometric product of different basis vectors
# Note that the inputs get sorted and repeated inputs are trimmed to 1 copy
e1e2e3 = BasisBlade(1,2,3)
print(e1e2e3)

e1e2e3


In [7]:
# Can make weighted blades by scalar multiplication against BasisBlade types 
# For simplicity, the BasisBlade type does not carry any weights
a = 2 * BasisBlade(None,) # handles scalars
b = 3.14 * BasisBlade(1,2,3)
c = 2 * BasisBlade(3)
print(f"{a}, {b}, {c}, type:{type(a)}")

2 (1), 3.14 e1e2e3, 2 e3, type:<class '__main__.Blade'>


In [8]:
# To handle inputs of specifically ordered basis vector indices
# use a different function that returns a Blade object to carry the sign
d = OrdBlade(2,1,3,2,0,1)
print(f"{d}  with type {type(d)}")

-1 e0e3  with type <class '__main__.Blade'>


In [9]:
print(f"Handles repeated basis vectors well")
print(f"{OrdBlade(1,2)}; {OrdBlade(1,1,2)}; {OrdBlade(1,1,1,2)}; {OrdBlade(1,2,1)}")

Handles repeated basis vectors well
1 e1e2; 1 e2; 1 e1e2; -1 e2


In [10]:
print("Scalar and blade-blade multiplication are implemented")
print(5*BasisBlade(1,)*3*BasisBlade(1,2)*2)
print()
print("Blade-blade addition is supported and gives a MultivectorExplicit object\n"
      "which basically stores a list of BasisBlades and the weights/coefficients")
print(a+b)

Scalar and blade-blade multiplication are implemented
30 e2

Blade-blade addition is supported and gives a MultivectorExplicit object
which basically stores a list of BasisBlades and the weights/coefficients
2 + 3.14 e1e2e3


In [11]:
print("Another blade-blade addition involving subtraction\n"
      "one of the operations collapses the two blades into a single blade\n"
      "the second operation creates a MultivectorExplicit because they don't share the basis blade\n")
print(4*OrdBlade(None) + (2*OrdBlade(1,2) - 1*OrdBlade(2,1)))

Another blade-blade addition involving subtraction
one of the operations collapses the two blades into a single blade
the second operation creates a MultivectorExplicit because they don't share the basis blade

4 + 3 e1e2


In [12]:
# Can handle different orders of the blades
print(OrdBlade(1,2) + 3 + 2*OrdBlade(2,1) + OrdBlade(1) + OrdBlade(1,3,2) + OrdBlade(2))

3 + 1 e1 + 1 e2 - 1 e1e2 - 1 e1e2e3


In [13]:
term_a = (1+OrdBlade(1,2) + OrdBlade(1,2,3))
term_b = (1+OrdBlade(1)+OrdBlade(3))
print(f"Result of the example from class: {term_a * term_b}")

Result of the example from class: 1 + 1 e1 - 1 e2 + 1 e3 + 2 e1e2 + 1 e2e3 + 2 e1e2e3


In [14]:
# Now get to the flattened representation

def get_basis_set(n):
    """Get the basis of G^n using the powerset of {1,2, ..., n}"""
    basis_set = []
    for subset in powerset(range(1,1+n)):
        basis_set.append(BasisBlade(*subset))
    return basis_set

def basis_interaction_table(n, print_table=False):
    """Find all the pairwise interactions of the basis"""
    basis_set = get_basis_set(n)
    table = [[b1*b2 for b2 in basis_set] for b1 in basis_set]
    col_width = 2 + 2*n
    if print_table:
        for row in table:
            for entry in row:
                pm = '+' if np.sign(entry.weight) >=0  else '-'
                print(f"{pm+str(entry.basis_blade):>{col_width}}", end=" ")
            print()        
    return table

def basis_dense_index_map(n):
    """Get a dense mapping from tuples of basis vectors used to indices"""
    index_map = {tuple(bv.idcs): idx for idx, bv in enumerate(get_basis_set(n))}
    return {None: 0, **index_map} # Just explicitly catch the None case

def lookup_table_2d(n):
    """Go from input entry index to the output index after the geometric product"""
    idx_map = basis_dense_index_map(n)
    table   = basis_interaction_table(n)
    lookup_table = np.array([[idx_map[tuple(bv.basis_blade.idcs)] for bv in row] for row in table])
    sign_table   = np.array([[np.sign(bv.weight) for bv in row] for row in table])
    return lookup_table, sign_table

class GnObj():
    """Object to hold the lookup table for a given dimensionality n"""
    def __init__(self, n):
        self.n = n
        self.blen = 2**n # basis length
        self.powerset_out = list(powerset(range(0,n)))
        self.basis_set = get_basis_set(n)
        self.idx_map   = basis_dense_index_map(n)
        lookup_table, sign_table = lookup_table_2d(n)
        self.lookup_table = lookup_table
        self.sign_table   = sign_table

        _, _, idxk = np.indices((2**n, 2**n, 2**n))
        self.dense_lookup_table = self.lookup_table[:, :, np.newaxis] == idxk

    def flatten_mve(self, mvec):
        """Takes a MultivectorExplicit and sends it into the flattened representation using the indices"""
        out_arr = np.zeros(2**self.n)
        for bv in mvec.blades:
            out_arr[self.idx_map[bv.hashable_idcs()]] = bv.weight
        return out_arr
    
    def to_mve(self, mv1):
        """Blow up an array to the Multivector Explicit form"""
        blade_list = []
        for b_idcs in self.powerset_out:
            idx = self.idx_map[b_idcs]
            new_blade = Blade(mv1[idx], self.basis_set[idx])
            if new_blade.weight != 0:
                blade_list.append(new_blade)
        return MultivectorExplicit(blade_list)

    def mult_dense(self, mv1, mv2):
        """Takes multivectors in flattened form"""
        assert(len(mv1) == self.blen)
        assert(len(mv2) == self.blen)
        # First, get the pairwise products
        init_mult_table = mv1[:, np.newaxis] * mv2[np.newaxis, :] * self.sign_table
        # Next, need to map to the outputs using the sign table
        flattened_vals = np.sum(self.dense_lookup_table * init_mult_table[:,:, np.newaxis], axis=(0,1))
        return flattened_vals

In [15]:
print(f"Flattening operation onto the MultivectorExplicit objects")
GnObj(3).flatten_mve(OrdBlade(1,2,3)+OrdBlade(1,2)-2*OrdBlade(1))

Flattening operation onto the MultivectorExplicit objects


array([ 0., -2.,  0.,  0.,  1.,  0.,  0.,  1.])

In [16]:
# term_a = (1+OrdBlade(1,2) + OrdBlade(1,2,3))
# term_b = (1+OrdBlade(1)+OrdBlade(3))
G3Obj = GnObj(3)
flattened_a = G3Obj.flatten_mve(term_a)
flattened_b = G3Obj.flatten_mve(term_b)
print(f"Two different ways to do the same multiplication represented in two different ways")
print(f"Explicit multiplication:  {G3Obj.flatten_mve(term_a * term_b)}")
print(f"Flattened multiplication: {G3Obj.mult_dense(flattened_a, flattened_b)}")
print(f"From Explicit:  {term_a * term_b}")
print(f"From flattened: {G3Obj.to_mve(G3Obj.mult_dense(flattened_a, flattened_b))}")

Two different ways to do the same multiplication represented in two different ways
Explicit multiplication:  [ 1.  1. -1.  1.  2.  0.  1.  2.]
Flattened multiplication: [ 1.  1. -1.  1.  2.  0.  1.  2.]
From Explicit:  1 + 1 e1 - 1 e2 + 1 e3 + 2 e1e2 + 1 e2e3 + 2 e1e2e3
From flattened: 1.0 + 1.0 e1 - 1.0 e2 + 1.0 e3 + 2.0 e1e2 + 1.0 e2e3 + 2.0 e1e2e3


In [17]:
print(f"Lookup table by indices -- might be helpful for looking for patterns")
print(lookup_table_2d(3)[0])

Lookup table by indices -- might be helpful for looking for patterns
[[0 1 2 3 4 5 6 7]
 [1 0 4 5 2 3 7 6]
 [2 4 0 6 1 7 3 5]
 [3 5 6 0 7 1 2 4]
 [4 2 1 7 0 6 5 3]
 [5 3 7 1 6 0 4 2]
 [6 7 3 2 5 4 0 1]
 [7 6 5 4 3 2 1 0]]


In [18]:
print(f"Full interaction table in terms of the signed basis blades")
table3 = basis_interaction_table(3, print_table=True)

Full interaction table in terms of the signed basis blades
    +(1)      +e1      +e2      +e3    +e1e2    +e1e3    +e2e3  +e1e2e3 
     +e1     +(1)    +e1e2    +e1e3      +e2      +e3  +e1e2e3    +e2e3 
     +e2    -e1e2     +(1)    +e2e3      -e1  -e1e2e3      +e3    -e1e3 
     +e3    -e1e3    -e2e3     +(1)  +e1e2e3      -e1      -e2    +e1e2 
   +e1e2      -e2      +e1  +e1e2e3     -(1)    -e2e3    +e1e3      -e3 
   +e1e3      -e3  -e1e2e3      +e1    +e2e3     -(1)    -e1e2      +e2 
   +e2e3  +e1e2e3      -e3      +e2    -e1e3    +e1e2     -(1)      -e1 
 +e1e2e3    +e2e3    -e1e3    +e1e2      -e3      +e2      -e1     -(1) 


In [19]:
# # In case you want to see wider tables as HTML tables...
# import pandas as pd
# from IPython.display import HTML
# table = basis_interaction_table(3)
# HTML(pd.DataFrame(table).to_html(index=False,header=False))